### Missing Not At Random

In [1]:
import os
os.chdir("..")
import numpy as np
import pandas as pd

In [2]:
from utils.data import create_compas_dataset, create_adult_dataset, create_titanic_dataset, create_communities_dataset, create_german_dataset, create_bank_dataset

In [17]:
data_complete = create_bank_dataset()
tmp_concat = pd.concat([data_complete.X, pd.DataFrame(data_complete.y, columns=["_TARGET_"])], axis=1)
tmp_concat.dropna(inplace=True)
tmp_concat.reset_index(drop=True, inplace=True)
data_complete.X = tmp_concat.drop(columns=["_TARGET_"]).copy()
data_complete.y = tmp_concat["_TARGET_"].copy().to_numpy().ravel()
X = data_complete.X.copy().drop(columns=data_complete.protected_features)
X.head()

,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3
1,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3
2,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3
3,1,1,3,0,1506,1,0,2,5,8,92,1,-1,0,3
4,11,2,3,0,1,0,0,2,5,8,198,1,-1,0,3


In [4]:
X.shape

(32561, 9)

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

def convert_multiple_features(K_df, range_min, range_max, scalar_0=0.05, scalar_1=2):
    max_iter = 50
    while(max_iter > 0):
        scalars = np.random.standard_normal(K_df.shape) * scalar_0
        scalar_a = np.random.standard_normal() - scalar_1
        K = K_df.to_numpy()
        K = scaler.fit_transform(K)
        M = scalar_a + (scalars * K)
        p = 1 / (1 + np.exp(-M))
        ratio = p.mean(axis=0)
        if ratio.min() >= range_min and ratio.max() <= range_max:
            break
        max_iter -= 1
    if max_iter <= 0:
        return None
    p_bi = np.random.binomial(1, p).astype(np.bool)
    K_df = K_df.where(~p_bi, other=np.nan)
    return K_df

In [ ]:
for i in range(100):
    K_df = convert_multiple_features(X, range_min=0.1, range_max=0.3)
    assert K_df is not None
    print(K_df.isnull().sum().sum() / X.size)

0.1381418976207855
0.15914121194694505
0.2154682120133006
0.11896219946473204
0.12490912978404961
0.11813791628880876
0.2233409273554371
0.16096230268444994
0.270151069430006
0.19758613316818177
